<a href="https://colab.research.google.com/github/swampxx/ML-algorithms-on-ids-datasets/blob/master/ids2018_thursday_preprocessing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [61]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [62]:
import numpy as np
import pandas as pd

from sklearn.kernel_approximation import RBFSampler
from sklearn.linear_model import SGDClassifier
from sklearn import svm
from sklearn.metrics import classification_report
from sklearn import metrics
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import (precision_score, recall_score, f1_score, accuracy_score, mean_squared_error, mean_absolute_error)
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import Normalizer
from sklearn.model_selection import GridSearchCV
from sklearn.feature_selection import SelectFromModel, SelectKBest
from sklearn.feature_selection import chi2, f_classif
from sklearn.svm import SVC
from sklearn.metrics import confusion_matrix
from sklearn.metrics import (precision_score, recall_score,f1_score, accuracy_score,mean_squared_error,mean_absolute_error, roc_curve, classification_report,auc)


In [63]:
!ls drive/My\ Drive/security_term_project/

path = "drive/My Drive/security_term_project/"

66398.pdf			      test_y_processed_fi.txt
ids_2018_thursday.csv		      test_y_processed_kbest.txt
ids2018-thursday_final_df_labels.txt  train_x_processed_fi.txt
ids2018-thursday_final_df.txt	      train_x_processed_kbest.txt
test_x_processed_fi.txt		      train_y_processed_fi.txt
test_x_processed_kbest.txt	      train_y_processed_kbest.txt


In [64]:
df = pd.read_csv(path+"ids_2018_thursday.csv")

In [65]:
df.head()

,Dst Port,Protocol,Timestamp,Flow Duration,Tot Fwd Pkts,Tot Bwd Pkts,TotLen Fwd Pkts,TotLen Bwd Pkts,Fwd Pkt Len Max,Fwd Pkt Len Min,Fwd Pkt Len Mean,Fwd Pkt Len Std,Bwd Pkt Len Max,Bwd Pkt Len Min,Bwd Pkt Len Mean,Bwd Pkt Len Std,Flow Byts/s,Flow Pkts/s,Flow IAT Mean,Flow IAT Std,Flow IAT Max,Flow IAT Min,Fwd IAT Tot,Fwd IAT Mean,Fwd IAT Std,Fwd IAT Max,Fwd IAT Min,Bwd IAT Tot,Bwd IAT Mean,Bwd IAT Std,Bwd IAT Max,Bwd IAT Min,Fwd PSH Flags,Bwd PSH Flags,Fwd URG Flags,Bwd URG Flags,Fwd Header Len,Bwd Header Len,Fwd Pkts/s,Bwd Pkts/s,Pkt Len Min,Pkt Len Max,Pkt Len Mean,Pkt Len Std,Pkt Len Var,FIN Flag Cnt,SYN Flag Cnt,RST Flag Cnt,PSH Flag Cnt,ACK Flag Cnt,URG Flag Cnt,CWE Flag Count,ECE Flag Cnt,Down/Up Ratio,Pkt Size Avg,Fwd Seg Size Avg,Bwd Seg Size Avg,Fwd Byts/b Avg,Fwd Pkts/b Avg,Fwd Blk Rate Avg,Bwd Byts/b Avg,Bwd Pkts/b Avg,Bwd Blk Rate Avg,Subflow Fwd Pkts,Subflow Fwd Byts,Subflow Bwd Pkts,Subflow Bwd Byts,Init Fwd Win Byts,Init Bwd Win Byts,Fwd Act Data Pkts,Fwd Seg Size Min,Active Mean,Active Std,Active Max,Active Min,Idle Mean,Idle Std,Idle Max,Idle Min,Label
0,0,0,15/02/2018 08:25:18,112641158,3,0,0,0,0,0,0.000000,0.000000,0,0,0.000000,0.000000,0.000000,0.026633,56320579.00,7.042784e+02,56321077,56320081,112641158,5.632058e+07,7.042784e+02,56321077,56320081,0,0.000000e+00,0.000000e+00,0,0,0,0,0,0,0,0,0.026633,0.000000,0,0,0.000000,0.000000,0.000000,0,0,0,0,0,0,0,0,0,0.0,0.000000,0.000000,0,0,0,0,0,0,3,0,0,0,-1,-1,0,0,0.0,0.000000,0,0,56320579.0,7.042784e+02,56321077,56320081,Benign
1,22,6,15/02/2018 08:29:05,37366762,14,12,2168,2993,712,0,154.857143,254.855270,976,0,249.416667,395.929392,138.117400,0.695806,1494670.48,3.894924e+06,15617415,7,37366762,2.874366e+06,5.104444e+06,15617415,40,37366730,3.396975e+06,5.564224e+06,15936762,893,0,0,0,0,456,416,0.374665,0.321141,0,976,191.148148,320.122898,102478.669516,0,0,0,1,0,0,0,0,0,198.5,154.857143,249.416667,0,0,0,0,0,0,14,2168,12,2993,29200,232,8,32,1024353.0,649038.754495,1601183,321569,11431221.0,3.644991e+06,15617415,8960247,Benign
2,47514,6,15/02/2018 08:29:42,543,2,0,64,0,64,0,32.000000,45.254834,0,0,0.000000,0.000000,117863.720074,3683.241252,543.00,0.000000e+00,543,543,543,5.430000e+02,0.000000e+00,543,543,0,0.000000e+00,0.000000e+00,0,0,1,0,0,0,64,0,3683.241252,0.000000,0,64,42.666667,36.950417,1365.333333,0,1,0,0,1,0,0,0,0,64.0,32.000000,0.000000,0,0,0,0,0,0,2,64,0,0,244,-1,0,32,0.0,0.000000,0,0,0.0,0.000000e+00,0,0,Benign
3,0,0,15/02/2018 08:28:07,112640703,3,0,0,0,0,0,0.000000,0.000000,0,0,0.000000,0.000000,0.000000,0.026633,56320351.50,3.669884e+02,56320611,56320092,112640703,5.632035e+07,3.669884e+02,56320611,56320092,0,0.000000e+00,0.000000e+00,0,0,0,0,0,0,0,0,0.026633,0.000000,0,0,0.000000,0.000000,0.000000,0,0,0,0,0,0,0,0,0,0.0,0.000000,0.000000,0,0,0,0,0,0,3,0,0,0,-1,-1,0,0,0.0,0.000000,0,0,56320351.5,3.669884e+02,56320611,56320092,Benign
4,0,0,15/02/2018 08:30:56,112640874,3,0,0,0,0,0,0.000000,0.000000,0,0,0.000000,0.000000,0.000000,0.026633,56320437.00,7.198347e+02,56320946,56319928,112640874,5.632044e+07,7.198347e+02,56320946,56319928,0,0.000000e+00,0.000000e+00,0,0,0,0,0,0,0,0,0.026633,0.000000,0,0,0.000000,0.000000,0.000000,0,0,0,0,0,0,0,0,0,0.0,0.000000,0.000000,0,0,0,0,0,0,3,0,0,0,-1,-1,0,0,0.0,0.000000,0,0,56320437.0,7.198347e+02,56320946,56319928,Benign


In [66]:
df.dropna(inplace=True)
df["Flow Pkts/s"] = pd.to_numeric(df["Flow Pkts/s"], errors='coerce')
df.drop('Timestamp', axis=1, inplace=True)
df['Protocol'] = df['Protocol'].astype("category")
one_hot = pd.get_dummies(df["Protocol"], prefix="Protocol")
df = df.drop("Protocol", axis=1)
df = df.join(one_hot)
df = df[~df.isin([np.nan, np.inf, -np.inf]).any(1)]
df = df.replace("Benign", 0).replace('DoS attacks-GoldenEye', 1).replace('DoS attacks-Slowloris', 1)
df.info(verbose=True)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1040548 entries, 0 to 1048574
Data columns (total 81 columns):
 #   Column             Non-Null Count    Dtype  
---  ------             --------------    -----  
 0   Dst Port           1040548 non-null  int64  
 1   Flow Duration      1040548 non-null  int64  
 2   Tot Fwd Pkts       1040548 non-null  int64  
 3   Tot Bwd Pkts       1040548 non-null  int64  
 4   TotLen Fwd Pkts    1040548 non-null  int64  
 5   TotLen Bwd Pkts    1040548 non-null  int64  
 6   Fwd Pkt Len Max    1040548 non-null  int64  
 7   Fwd Pkt Len Min    1040548 non-null  int64  
 8   Fwd Pkt Len Mean   1040548 non-null  float64
 9   Fwd Pkt Len Std    1040548 non-null  float64
 10  Bwd Pkt Len Max    1040548 non-null  int64  
 11  Bwd Pkt Len Min    1040548 non-null  int64  
 12  Bwd Pkt Len Mean   1040548 non-null  float64
 13  Bwd Pkt Len Std    1040548 non-null  float64
 14  Flow Byts/s        1040548 non-null  float64
 15  Flow Pkts/s        1040548 non-n

## Feature Selection

In [67]:
all_labels = df.Label.copy().values
all_data = df.drop('Label', axis = 1).values

In [68]:
all_labels.shape, all_data.shape

((1040548,), (1040548, 80))

### With KBest

In [69]:
featureSelector = SelectKBest(f_classif, k=40)
featureSelector.fit(all_data, all_labels)

/usr/local/lib/python3.6/dist-packages/sklearn/feature_selection/_univariate_selection.py:114: UserWarning: Features [31 32 33 49 55 56 57 58 59 60] are constant.
  UserWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/feature_selection/_univariate_selection.py:115: RuntimeWarning: invalid value encountered in true_divide
  f = msb / msw


SelectKBest(k=40, score_func=<function f_classif at 0x7f576ab96ae8>)

In [70]:
df_withoutLabel =  df.drop('Label', axis = 1)
features = df_withoutLabel.columns[featureSelector.get_support()]

In [71]:
features

Index(['Dst Port', 'Flow Duration', 'Fwd Pkt Len Max', 'Fwd Pkt Len Min',
       'Fwd Pkt Len Mean', 'Fwd Pkt Len Std', 'Bwd Pkt Len Min',
       'Bwd Pkt Len Std', 'Flow IAT Mean', 'Flow IAT Std', 'Flow IAT Max',
       'Fwd IAT Mean', 'Fwd IAT Max', 'Fwd IAT Min', 'Bwd IAT Tot',
       'Bwd IAT Mean', 'Bwd IAT Std', 'Bwd IAT Max', 'Bwd IAT Min',
       'Pkt Len Min', 'Pkt Len Max', 'Pkt Len Std', 'RST Flag Cnt',
       'PSH Flag Cnt', 'ECE Flag Cnt', 'Down/Up Ratio', 'Fwd Seg Size Avg',
       'Init Fwd Win Byts', 'Init Bwd Win Byts', 'Fwd Seg Size Min',
       'Active Mean', 'Active Std', 'Active Max', 'Active Min', 'Idle Mean',
       'Idle Std', 'Idle Max', 'Idle Min', 'Protocol_6', 'Protocol_17'],
      dtype='object')

### With Feature Importance RandomForestClassifier

In [72]:
fs = SelectFromModel(RandomForestClassifier())
# learn relationship from training data
fs.fit(all_data, all_labels)

SelectFromModel(estimator=RandomForestClassifier(bootstrap=True, ccp_alpha=0.0,
                                                 class_weight=None,
                                                 criterion='gini',
                                                 max_depth=None,
                                                 max_features='auto',
                                                 max_leaf_nodes=None,
                                                 max_samples=None,
                                                 min_impurity_decrease=0.0,
                                                 min_impurity_split=None,
                                                 min_samples_leaf=1,
                                                 min_samples_split=2,
                                                 min_weight_fraction_leaf=0.0,
                                                 n_estimators=100, n_jobs=None,
                                                 oob_score=False,

In [73]:
features3 = df_withoutLabel.columns[fs.get_support()]

In [74]:
features3

Index(['Dst Port', 'Flow Duration', 'Tot Bwd Pkts', 'Flow Pkts/s',
       'Flow IAT Mean', 'Flow IAT Max', 'Flow IAT Min', 'Fwd IAT Mean',
       'Fwd IAT Max', 'Bwd IAT Mean', 'Bwd IAT Max', 'Fwd Header Len',
       'Bwd Header Len', 'Fwd Pkts/s', 'Pkt Len Mean', 'Init Fwd Win Byts',
       'Fwd Seg Size Min'],
      dtype='object')

### With Feature Importance LogisticRegression

Failed.....

In [75]:
fs1 = SelectFromModel(LogisticRegression())
fs1.fit(all_data, all_labels)

/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


SelectFromModel(estimator=LogisticRegression(C=1.0, class_weight=None,
                                             dual=False, fit_intercept=True,
                                             intercept_scaling=1, l1_ratio=None,
                                             max_iter=100, multi_class='auto',
                                             n_jobs=None, penalty='l2',
                                             random_state=None, solver='lbfgs',
                                             tol=0.0001, verbose=0,
                                             warm_start=False),
                max_features=None, norm_order=1, prefit=False, threshold=None)

In [76]:
features2 = df_withoutLabel.columns[fs1.get_support()]

In [77]:
features2

Index(['Dst Port', 'Flow Duration', 'Flow Byts/s', 'Flow Pkts/s',
       'Flow IAT Max', 'Fwd IAT Tot', 'Fwd IAT Mean', 'Fwd IAT Std',
       'Fwd IAT Max', 'Fwd IAT Min', 'Bwd IAT Tot', 'Bwd IAT Mean',
       'Bwd IAT Std', 'Bwd IAT Max', 'Bwd IAT Min', 'Fwd Pkts/s',
       'Pkt Len Var', 'Active Min', 'Idle Mean', 'Idle Std', 'Idle Max'],
      dtype='object')

### Selected Features

In [78]:
selected_columns = ['Dst Port', 'Flow Duration', 'Fwd Pkt Len Max', 'Fwd Pkt Len Min',
       'Fwd Pkt Len Mean', 'Fwd Pkt Len Std', 'Bwd Pkt Len Min',
       'Bwd Pkt Len Std', 'Flow IAT Mean', 'Flow IAT Std', 'Flow IAT Max',
       'Fwd IAT Mean', 'Fwd IAT Max', 'Fwd IAT Min', 'Bwd IAT Tot',
       'Bwd IAT Mean', 'Bwd IAT Std', 'Bwd IAT Max', 'Bwd IAT Min',
       'Pkt Len Min', 'Pkt Len Max', 'Pkt Len Std', 'RST Flag Cnt',
       'PSH Flag Cnt', 'ECE Flag Cnt', 'Down/Up Ratio', 'Fwd Seg Size Avg',
       'Init Fwd Win Byts', 'Init Bwd Win Byts', 'Fwd Seg Size Min',
       'Active Mean', 'Active Std', 'Active Max', 'Active Min', 'Idle Mean',
       'Idle Std', 'Idle Max', 'Idle Min', 'Protocol_6', 'Protocol_17']
       

In [79]:
selected_columns += ['Dst Port', 'Flow Duration', 'Tot Bwd Pkts', 'TotLen Bwd Pkts',
       'Flow Pkts/s', 'Flow IAT Mean', 'Flow IAT Std', 'Flow IAT Max',
       'Flow IAT Min', 'Fwd IAT Tot', 'Fwd IAT Mean', 'Fwd IAT Max',
       'Fwd Header Len', 'Fwd Pkts/s', 'Subflow Bwd Pkts', 'Subflow Bwd Byts',
       'Init Fwd Win Byts', 'Init Bwd Win Byts', 'Fwd Seg Size Min', 'URG Flag Cnt',
       'SYN Flag Cnt', 'RST Flag Cnt', 'PSH Flag Cnt', 'Pkt Size Avg', 'Protocol_0',
       'Flow Pkts/s', 'FIN Flag Cnt', 'ECE Flag Cnt',  'ACK Flag Cnt']

In [80]:
res_feature_list = list(set(selected_columns))

In [81]:
print(len(res_feature_list))
print(res_feature_list)

55
['Fwd Pkt Len Std', 'Bwd IAT Mean', 'Subflow Bwd Pkts', 'Fwd IAT Max', 'Pkt Len Max', 'Bwd IAT Std', 'Flow Pkts/s', 'Active Max', 'Flow IAT Mean', 'Init Fwd Win Byts', 'PSH Flag Cnt', 'Bwd IAT Min', 'Protocol_6', 'Bwd IAT Tot', 'Flow Duration', 'Active Min', 'URG Flag Cnt', 'Fwd IAT Mean', 'SYN Flag Cnt', 'ACK Flag Cnt', 'Fwd Pkt Len Max', 'Fwd Seg Size Avg', 'Active Mean', 'Idle Max', 'Fwd IAT Tot', 'Pkt Size Avg', 'Down/Up Ratio', 'Active Std', 'Fwd Pkt Len Mean', 'TotLen Bwd Pkts', 'Subflow Bwd Byts', 'RST Flag Cnt', 'Idle Std', 'Dst Port', 'Tot Bwd Pkts', 'Pkt Len Min', 'Flow IAT Min', 'Pkt Len Std', 'Bwd Pkt Len Min', 'Protocol_0', 'Fwd Seg Size Min', 'FIN Flag Cnt', 'Idle Min', 'Bwd IAT Max', 'ECE Flag Cnt', 'Idle Mean', 'Fwd IAT Min', 'Flow IAT Max', 'Fwd Pkt Len Min', 'Fwd Pkts/s', 'Init Bwd Win Byts', 'Fwd Header Len', 'Protocol_17', 'Flow IAT Std', 'Bwd Pkt Len Std']


In [82]:
final_df = df.drop('Label', axis = 1)
final_df = final_df[res_feature_list]
final_df.info(verbose=True)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1040548 entries, 0 to 1048574
Data columns (total 55 columns):
 #   Column             Non-Null Count    Dtype  
---  ------             --------------    -----  
 0   Fwd Pkt Len Std    1040548 non-null  float64
 1   Bwd IAT Mean       1040548 non-null  float64
 2   Subflow Bwd Pkts   1040548 non-null  int64  
 3   Fwd IAT Max        1040548 non-null  int64  
 4   Pkt Len Max        1040548 non-null  int64  
 5   Bwd IAT Std        1040548 non-null  float64
 6   Flow Pkts/s        1040548 non-null  float64
 7   Active Max         1040548 non-null  int64  
 8   Flow IAT Mean      1040548 non-null  float64
 9   Init Fwd Win Byts  1040548 non-null  int64  
 10  PSH Flag Cnt       1040548 non-null  int64  
 11  Bwd IAT Min        1040548 non-null  int64  
 12  Protocol_6         1040548 non-null  uint8  
 13  Bwd IAT Tot        1040548 non-null  int64  
 14  Flow Duration      1040548 non-null  int64  
 15  Active Min         1040548 non-n

In [83]:
final_df_vals = final_df.values

In [84]:
#np.savetxt(path+"ids2018-thursday_final_df.txt", final_df_vals)
#np.savetxt(path+"ids2018-thursday_final_df_labels.txt", all_labels)